In [ ]:
import cv2
import os
import zipfile
from google.colab import files

# Upload the video files
uploaded = files.upload()

# Process each uploaded file
for video_path in uploaded.keys():
    base_filename = os.path.splitext(video_path)[0]

    # Create a directory to store frames for this video
    frames_dir = f'frames_{base_filename}'
    os.makedirs(frames_dir, exist_ok=True)

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Check if video opened successfully
    if not cap.isOpened():
        print(f"Error: Could not open video {video_path}.")
        continue
    else:
        frame_count = 0
        saved_frame_count = 0

        while True:
            ret, frame = cap.read()

            if not ret:
                break

            # Save every 30th frame
            if frame_count % 30 == 0:
                frame_path = os.path.join(frames_dir, f'frame_{saved_frame_count:04d}.jpg')
                cv2.imwrite(frame_path, frame)
                saved_frame_count += 1

            frame_count += 1

        cap.release()

    # Zip the frames directory using the base file name
    zip_filename = f'{base_filename}_frames.zip'
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for root, dirs, files_in_dir in os.walk(frames_dir): # Change 'files' to 'files_in_dir'
            for file in files_in_dir:
                zipf.write(os.path.join(root, file),
                           os.path.relpath(os.path.join(root, file),
                                           os.path.join(frames_dir, '..')))

    # Clean up frames directory
    for file in os.listdir(frames_dir):
        os.remove(os.path.join(frames_dir, file))
    os.rmdir(frames_dir)

    # Download the zip file
    files.download(zip_filename) # Use the original 'files' object for download